## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Sargatskoye,55.6101,73.4972,11.08,95,99,13.49,RU,overcast clouds
1,1,Oussouye,12.4850,-16.5469,68.43,22,52,7.43,SN,broken clouds
2,2,Punta Arenas,-53.1500,-70.9167,48.20,71,0,20.71,CL,clear sky
3,3,Saint George,37.1041,-113.5841,39.20,75,1,3.06,US,clear sky
4,4,Katubao,5.8892,124.8294,86.00,58,75,10.36,PH,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
print(min_temp,max_temp)

70.0 90.0


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
4,4,Katubao,5.8892,124.8294,86.00,58,75,10.36,PH,broken clouds
6,6,Avarua,-21.2078,-159.7750,78.80,89,75,2.30,CK,broken clouds
10,10,Port Alfred,-33.5906,26.8910,73.99,79,42,2.98,ZA,scattered clouds
14,14,Auki,-8.7676,160.7034,83.32,75,95,15.52,SB,light rain
15,15,Faanui,-16.4833,-151.7500,82.45,70,98,10.94,PF,overcast clouds
18,18,Hermanus,-34.4187,19.2345,70.00,79,2,1.01,ZA,clear sky
19,19,Saint-Pierre,-21.3393,55.4781,87.80,62,0,18.41,RE,clear sky
20,20,Port Blair,11.6667,92.7500,83.57,65,0,11.77,IN,clear sky
21,21,Victoria,22.2855,114.1577,73.40,40,0,9.22,HK,clear sky
23,23,Meulaboh,4.1363,96.1285,85.12,62,68,5.41,ID,broken clouds


In [5]:
len(preferred_cities_df)

248

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                4
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [8]:
# NA is dropped
len(clean_df)

244

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Katubao,PH,86.00,broken clouds,5.8892,124.8294,
6,Avarua,CK,78.80,broken clouds,-21.2078,-159.7750,
10,Port Alfred,ZA,73.99,scattered clouds,-33.5906,26.8910,
14,Auki,SB,83.32,light rain,-8.7676,160.7034,
15,Faanui,PF,82.45,overcast clouds,-16.4833,-151.7500,
18,Hermanus,ZA,70.00,clear sky,-34.4187,19.2345,
19,Saint-Pierre,RE,87.80,clear sky,-21.3393,55.4781,
20,Port Blair,IN,83.57,clear sky,11.6667,92.7500,
21,Victoria,HK,73.40,clear sky,22.2855,114.1577,
23,Meulaboh,ID,85.12,broken clouds,4.1363,96.1285,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Katubao,PH,86.00,broken clouds,5.8892,124.8294,IML Ecopark
6,Avarua,CK,78.80,broken clouds,-21.2078,-159.7750,Paradise Inn
10,Port Alfred,ZA,73.99,scattered clouds,-33.5906,26.8910,The Halyards Hotel
14,Auki,SB,83.32,light rain,-8.7676,160.7034,Auki Motel
15,Faanui,PF,82.45,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [11]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))